# GTI771 - Apprentissage machine avancé

### Created: Thiago M. Paixão <br> Revised: Alessandro L. Koerich <br> Ver 1.0 <br> December 2020¶

## NB2 - Template Matching Dataset Simpsons: using the mean image as prototype

In this notebook, we will address the classification of characters from the TV serie "The Simpson" using the template matching technique. Instead of trying to match a query against every image in the training partition, the query is now compared against a representative prototype (mean image) of each class.

The notebook is divided into four parts:

- Setup
- Train-test partitioning
- Template matching-based classification using mean image
- Performance evaluation

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../')

import os
import glob
import csv
import random
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict

# from skimage import data
from skimage import io
from skimage.transform import resize

from utils import show, show_collection, load_simpsons_dataset, resize_dataset, classify_template_matching

In [ ]:
# change this to the path where the dataset is located in your filesystem
DATASET_PATH = '../data/Simpsons-Train-Valid'

## Load and resize the Simpsons dataset

As in the NB1, the dataset (partitioned into training and test splits) are loaded and resized. Loading and resize functionalities are, now, implemented in ``utils.py``:

In [ ]:
train_set, test_set = load_simpsons_dataset(DATASET_PATH)
train_set_resized = resize_dataset(train_set, output_shape=(256, 256))
test_set_resized = resize_dataset(test_set, output_shape=(256, 256))

## Building the prototypes (mean images)

The function bellow computes the mean image for each class in a dataset. The result is dictionary that maps from labels to images:

In [ ]:
def mean_image(dataset):
    zero_arr = np.zeros_like(dataset[0][0], dtype=np.float32)
    counter = defaultdict(lambda: 0)
    accumulator = defaultdict(lambda: zero_arr.copy())
    for image, label in dataset:
        counter[label] += 1
        accumulator[label] += image
    for label in accumulator:
        accumulator[label] = (accumulator[label] / counter[label])
    mean_images_dataset = [(image, label) for label, image in accumulator.items()]
    return mean_images_dataset

prototypes = mean_image(train_set_resized)

# show the mean images (prototypes)
labels = [label for _, label in prototypes]
images = [prototype for prototype, _ in prototypes]
show_collection(images, labels, scale=0.75)

## Template matching-based classification

In [ ]:
image_query, label_query = random.choice(test_set_resized)
image_result, label_result = classify_template_matching(image_query, prototypes)

titles = ['query = {}'.format(label_query), 'result = {}'.format(label_result)]
images = [image_query, image_result]
show_collection(images, titles, scale=0.5)

## Performance evaluation

Compute some performance metrics on the test set.

In [ ]:
# Evaluation metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

To use the mentioned ``sklearn`` metrics, we first encode the labels ('bart', 'homer', ...) to numeric values ranging from $0$ to $n_{classes} - 1$:

In [ ]:
from sklearn import preprocessing

# building encoder
labels_train = [label for _, label in train_set_resized]
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(labels_train)

# encoding the labels of the test set
labels_test = [label for _, label in test_set_resized]
y_true = label_encoder.transform(labels_test)

print('True labels')
for label, y in zip(labels_test, y_true):
    print('{} -> {}'.format(label, y))

Predicting the labels of the test set (remember that the test set has a single exemplar of each class):

In [ ]:
labels = []
for image_query, _ in test_set_resized:
    _, label = classify_template_matching(image_query, prototypes)
    labels.append(label)
y_pred = label_encoder.transform(labels)

print('Predicted labels')
for label, y in zip(labels_test, y_pred):
    print('{} -> {}'.format(label, y))

Now, we can compute the metrics:

In [ ]:
acc = accuracy_score(y_true, y_pred)
print('Correct classification rate for the training dataset = {:.2f}%'.format(100 * acc))

In [ ]:
confusion_matrix(y_true, y_pred)

In [ ]:
report = classification_report(y_true, y_pred)
print(report)

Observe the warning above. This is due to zero division in the F1 metric calculation, which occurs when precision and recall are simultaneoulsy zero.